In [1]:
import sqlite3
import pandas as pd

In [4]:
conexion = sqlite3.connect("../data/eda.db")
cursor = conexion.cursor()

In [6]:
query = """
        CREATE TABLE IF NOT EXISTS Empleados (
            id INT PRIMARY KEY,
            nombre TEXT,
            genero TEXT,
            edad INT
        )
        """
conexion.execute(query)
conexion.commit()